In [3]:
!echo $PATH

# https://www.kaggle.com/code/ridhoaryo/telco-churn-analysis-and-modeling

/opt/conda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin


In [1]:
%pip install graphviz

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.0/47.0 kB 4.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [32]:
%pip install category_encoders

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 kB 5.9 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [39]:
%pip install xgboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.9/255.9 MB 2.5 MB/s eta 0:00:0000:0100:02
Note: you may need to restart the kernel to use updated packages.


In [30]:
%pip install imbalanced-learn

Note: you may need to restart the kernel to use updated packages.


In [5]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
pd.set_option('display.max_columns', 500)

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.feature_selection import SelectPercentile, SelectKBest, RFE
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV, cross_val_score, RepeatedStratifiedKFold,\
StratifiedKFold
from sklearn.preprocessing import PolynomialFeatures, StandardScaler, OneHotEncoder
from sklearn.decomposition import PCA
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.metrics import classification_report, plot_confusion_matrix, precision_recall_curve, plot_precision_recall_curve, confusion_matrix
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import RandomOverSampler, SMOTE
import category_encoders as ce
from scipy import stats
import graphviz

import xgboost as xgb

ModuleNotFoundError: No module named 'imblearn'

In [25]:
df = pd.read_excel('Telco_customer_churn.xlsx')

In [4]:
%pip install openpyxl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.1/242.1 kB 3.5 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [3]:
df.head()

,CustomerID,Count,Country,State,City,Zip Code,Lat Long,Latitude,Longitude,Gender,...,Contract,Paperless Billing,Payment Method,Monthly Charges,Total Charges,Churn Label,Churn Value,Churn Score,CLTV,Churn Reason
0,3668-QPYBK,1,United States,California,Los Angeles,90003,"33.964131, -118.272783",33.964131,-118.272783,Male,...,Month-to-month,Yes,Mailed check,53.85,108.15,Yes,1,86,3239,Competitor made better offer
1,9237-HQITU,1,United States,California,Los Angeles,90005,"34.059281, -118.30742",34.059281,-118.307420,Female,...,Month-to-month,Yes,Electronic check,70.70,151.65,Yes,1,67,2701,Moved
2,9305-CDSKC,1,United States,California,Los Angeles,90006,"34.048013, -118.293953",34.048013,-118.293953,Female,...,Month-to-month,Yes,Electronic check,99.65,820.5,Yes,1,86,5372,Moved
3,7892-POOKP,1,United States,California,Los Angeles,90010,"34.062125, -118.315709",34.062125,-118.315709,Female,...,Month-to-month,Yes,Electronic check,104.80,3046.05,Yes,1,84,5003,Moved
4,0280-XJGEX,1,United States,California,Los Angeles,90015,"34.039224, -118.266293",34.039224,-118.266293,Male,...,Month-to-month,Yes,Bank transfer (automatic),103.70,5036.3,Yes,1,89,5340,Competitor had better devices


In [4]:
df.dtypes

CustomerID            object
Count                  int64
Country               object
State                 object
City                  object
Zip Code               int64
Lat Long              object
Latitude             float64
Longitude            float64
Gender                object
Senior Citizen        object
Partner               object
Dependents            object
Tenure Months          int64
Phone Service         object
Multiple Lines        object
Internet Service      object
Online Security       object
Online Backup         object
Device Protection     object
Tech Support          object
Streaming TV          object
Streaming Movies      object
Contract              object
Paperless Billing     object
Payment Method        object
Monthly Charges      float64
Total Charges         object
Churn Label           object
Churn Value            int64
Churn Score            int64
CLTV                   int64
Churn Reason          object
dtype: object

In [5]:
df["Phone Service"].unique()

array(['Yes', 'No'], dtype=object)

In [68]:
df.loc[(df["Total Charges"] == ' '), "Total Charges"] = 0

In [69]:
df["Total Charges"] = pd.to_numeric(df["Total Charges"])

In [70]:
df["Total Charges"].dtype

dtype('float64')

In [12]:
df.replace(' ', '_', regex=True, inplace=True)
df.head()

,CustomerID,Count,Country,State,City,Zip Code,Lat Long,Latitude,Longitude,Gender,...,Contract,Paperless Billing,Payment Method,Monthly Charges,Total Charges,Churn Label,Churn Value,Churn Score,CLTV,Churn Reason
0,3668-QPYBK,1,United_States,California,Los_Angeles,90003,"33.964131,_-118.272783",33.964131,-118.272783,Male,...,Month-to-month,Yes,Mailed_check,53.85,108.15,Yes,1,86,3239,Competitor_made_better_offer
1,9237-HQITU,1,United_States,California,Los_Angeles,90005,"34.059281,_-118.30742",34.059281,-118.307420,Female,...,Month-to-month,Yes,Electronic_check,70.70,151.65,Yes,1,67,2701,Moved
2,9305-CDSKC,1,United_States,California,Los_Angeles,90006,"34.048013,_-118.293953",34.048013,-118.293953,Female,...,Month-to-month,Yes,Electronic_check,99.65,820.50,Yes,1,86,5372,Moved
3,7892-POOKP,1,United_States,California,Los_Angeles,90010,"34.062125,_-118.315709",34.062125,-118.315709,Female,...,Month-to-month,Yes,Electronic_check,104.80,3046.05,Yes,1,84,5003,Moved
4,0280-XJGEX,1,United_States,California,Los_Angeles,90015,"34.039224,_-118.266293",34.039224,-118.266293,Male,...,Month-to-month,Yes,Bank_transfer_(automatic),103.70,5036.30,Yes,1,89,5340,Competitor_had_better_devices


In [71]:
X = df.drop(['CustomerID', 'Country', 'State', 'Lat Long', 'Churn Label', 'Churn Reason'], axis=1).copy()

In [72]:
y = df['Churn Value'].copy()

In [18]:
pd.get_dummies(X, columns=['Payment Method']).head()

,CustomerID,Count,Country,State,City,Zip Code,Lat Long,Latitude,Longitude,Gender,...,Monthly Charges,Total Charges,Churn Label,Churn Score,CLTV,Churn Reason,Payment Method_Bank_transfer_(automatic),Payment Method_Credit_card_(automatic),Payment Method_Electronic_check,Payment Method_Mailed_check
0,3668-QPYBK,1,United_States,California,Los_Angeles,90003,"33.964131,_-118.272783",33.964131,-118.272783,Male,...,53.85,108.15,Yes,86,3239,Competitor_made_better_offer,0,0,0,1
1,9237-HQITU,1,United_States,California,Los_Angeles,90005,"34.059281,_-118.30742",34.059281,-118.307420,Female,...,70.70,151.65,Yes,67,2701,Moved,0,0,1,0
2,9305-CDSKC,1,United_States,California,Los_Angeles,90006,"34.048013,_-118.293953",34.048013,-118.293953,Female,...,99.65,820.50,Yes,86,5372,Moved,0,0,1,0
3,7892-POOKP,1,United_States,California,Los_Angeles,90010,"34.062125,_-118.315709",34.062125,-118.315709,Female,...,104.80,3046.05,Yes,84,5003,Moved,0,0,1,0
4,0280-XJGEX,1,United_States,California,Los_Angeles,90015,"34.039224,_-118.266293",34.039224,-118.266293,Male,...,103.70,5036.30,Yes,89,5340,Competitor_had_better_devices,1,0,0,0


In [19]:
X.columns

Index(['CustomerID', 'Count', 'Country', 'State', 'City', 'Zip Code',
       'Lat Long', 'Latitude', 'Longitude', 'Gender', 'Senior Citizen',
       'Partner', 'Dependents', 'Tenure Months', 'Phone Service',
       'Multiple Lines', 'Internet Service', 'Online Security',
       'Online Backup', 'Device Protection', 'Tech Support', 'Streaming TV',
       'Streaming Movies', 'Contract', 'Paperless Billing', 'Payment Method',
       'Monthly Charges', 'Total Charges', 'Churn Label', 'Churn Score',
       'CLTV', 'Churn Reason'],
      dtype='object')

In [73]:
X_encoded = pd.get_dummies(X, columns=['City', 'Gender', 'Senior Citizen', 'Partner', 'Dependents', 'Phone Service', 'Multiple Lines', 
                                    'Internet Service', 'Online Security', 'Online Backup', 'Device Protection', 'Tech Support', 'Streaming TV',
                                    'Streaming Movies', 'Contract', 'Paperless Billing', 'Payment Method'])
X_encoded.head()

,Count,Zip Code,Latitude,Longitude,Tenure Months,Monthly Charges,Total Charges,Churn Value,Churn Score,CLTV,City_Acampo,City_Acton,City_Adelanto,City_Adin,City_Agoura Hills,City_Aguanga,City_Ahwahnee,City_Alameda,City_Alamo,City_Albany,City_Albion,City_Alderpoint,City_Alhambra,City_Aliso Viejo,City_Alleghany,City_Alpaugh,City_Alpine,City_Alta,City_Altadena,City_Alturas,City_Alviso,City_Amador City,City_Amboy,City_Anaheim,City_Anderson,City_Angels Camp,City_Angelus Oaks,City_Angwin,City_Annapolis,City_Antelope,City_Antioch,City_Anza,City_Apple Valley,City_Applegate,City_Aptos,City_Arbuckle,City_Arcadia,City_Arcata,City_Armona,City_Arnold,City_Aromas,City_Arroyo Grande,City_Artesia,City_Arvin,City_Atascadero,City_Atherton,City_Atwater,City_Auberry,City_Auburn,City_Avalon,City_Avenal,City_Avery,City_Avila Beach,City_Azusa,City_Badger,City_Baker,City_Bakersfield,City_Baldwin Park,City_Ballico,City_Bangor,City_Banning,City_Barstow,City_Bass Lake,City_Bayside,City_Beale Afb,City_Beaumont,City_Bell,City_Bella Vista,City_Bellflower,City_Belmont,City_Belvedere Tiburon,City_Ben Lomond,City_Benicia,City_Benton,City_Berkeley,City_Berry Creek,City_Bethel Island,City_Beverly Hills,City_Bieber,City_Big Bar,City_Big Bear City,City_Big Bear Lake,City_Big Bend,City_Big Creek,City_Big Oak Flat,City_Big Pine,City_Big Sur,City_Biggs,City_Biola,City_Birds Landing,City_Bishop,City_Blairsden Graeagle,City_Blocksburg,City_Bloomington,City_Blue Lake,City_Blythe,City_Bodega,City_Bodega Bay,City_Bodfish,City_Bolinas,City_Bonita,City_Bonsall,City_Boonville,City_Boron,City_Borrego Springs,City_Boulder Creek,City_Boulevard,City_Bradley,City_Branscomb,City_Brawley,City_Brea,City_Brentwood,City_Bridgeport,City_Bridgeville,City_Brisbane,City_Brookdale,City_Brooks,City_Browns Valley,City_Brownsville,City_Buellton,City_Buena Park,City_Burbank,City_Burlingame,City_Burney,City_Burnt Ranch,City_Burson,City_Butte City,City_Buttonwillow,City_Byron,City_Cabazon,City_Calabasas,City_Calexico,City_Caliente,City_California City,City_California Hot Springs,City_Calimesa,City_Calipatria,City_Calistoga,City_Callahan,City_Calpine,City_Camarillo,City_Cambria,City_Camino,City_Camp Nelson,City_Campbell,City_Campo,City_Campo Seco,City_Camptonville,City_Canby,City_Canoga Park,City_Cantua Creek,City_Canyon Country,City_Canyon Dam,City_Capay,City_Capistrano Beach,City_Capitola,City_Cardiff By The Sea,City_Carlotta,City_Carlsbad,City_Carmel,City_Carmel By The Sea,City_Carmel Valley,City_Carmichael,City_Carnelian Bay,City_Carpinteria,City_Carson,City_Caruthers,City_Casmalia,City_Caspar,City_Cassel,City_Castaic,City_Castella,City_Castro Valley,City_Castroville,City_Cathedral City,City_Catheys Valley,City_Cayucos,City_Cazadero,City_Cedar Glen,City_Cedarville,City_Ceres,City_Cerritos,City_Challenge,City_Chatsworth,City_Chester,City_Chico,City_Chilcoot,City_Chino,City_Chino Hills,City_Chowchilla,City_Chualar,City_Chula Vista,City_Citrus Heights,City_Claremont,City_Clarksburg,City_Clayton,City_Clearlake,City_Clearlake Oaks,City_Clements,City_Clio,City_Clipper Mills,City_Cloverdale,City_Clovis,City_Coachella,City_Coalinga,City_Coarsegold,City_Cobb,City_Coleville,City_Colfax,City_Colton,City_Columbia,City_Colusa,City_Comptche,City_Compton,City_Concord,City_Cool,City_Copperopolis,City_Corcoran,City_Corning,City_Corona,City_Corona Del Mar,City_Coronado,City_Corte Madera,City_Costa Mesa,City_Cotati,City_Cottonwood,City_Coulterville,City_Courtland,City_Covelo,City_Covina,City_Crescent City,City_Crescent Mills,City_Cressey,City_Crestline,City_Creston,City_Crockett,City_Crows Landing,City_Culver City,City_Cupertino,City_Cutler,...,City_Selma,City_Sequoia National Park,City_Shafter,City_Shandon,City_Shasta,City_Shasta Lake,City_Shaver Lake,City_Sheep Ranch,City_Sheridan,City_Sherman Oaks,City_Shingle Springs,City_Shingletown,City_Shoshone,City_Sierra City,City_Sierra Madre,City_Sierraville,City_Silverado,City_Simi Valley,City_Sloughhouse,City_Smartville,City_Smith River,City_Snelling,City_Soda Spri

In [22]:
y.unique()

array([1, 0])

In [23]:
sum(y)/len(y)

0.2653698707936959

In [80]:
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, random_state=42, stratify=y)

In [81]:
sum(y_train)/len(y_train)

0.2654297614539947

In [82]:
sum(y_test)/len(y_test)

0.26519023282226006

In [83]:
clf_xgb = xgb.XGBClassifier(objective='binary:logistic', missing=None, seed=42)
clf_xgb.fit(X_train,
            y_train,
            verbose=True,
            early_stopping_rounds=10,
            eval_metric='aucpr',
            eval_set=[(X_test, y_test)])

/opt/conda/lib/python3.10/site-packages/xgboost/sklearn.py:793: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/xgboost/sklearn.py:793: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[0]	validation_0-aucpr:1.00000
[1]	validation_0-aucpr:1.00000
[2]	validation_0-aucpr:1.00000
[3]	validation_0-aucpr:1.00000
[4]	validation_0-aucpr:1.00000
[5]	validation_0-aucpr:1.00000
[6]	validation_0-aucpr:1.00000
[7]	validation_0-aucpr:1.00000
[8]	validation_0-aucpr:1.00000
[9]	validation_0-aucpr:1.00000


XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
              missing=None, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=42,
              reg_alpha=0, reg_lambda=1, ...)

In [76]:
X_encoded['Total Charges']

0        108.15
1        151.65
2        820.50
3       3046.05
4       5036.30
         ...   
7038    1419.40
7039    1990.50
7040    7362.90
7041     346.45
7042    6844.50
Name: Total Charges, Length: 7043, dtype: float64

In [77]:
plot_confusion_matrix(clf_xgb, X_test, y_test, values_format='d', display_labels=["Did not leave", "Left"])

/opt/conda/lib/python3.10/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function plot_confusion_matrix is deprecated; Function `plot_confusion_matrix` is deprecated in 1.0 and will be removed in 1.2. Use one of the class methods: ConfusionMatrixDisplay.from_predictions or ConfusionMatrixDisplay.from_estimator.
  warnings.warn(msg, category=FutureWarning)


XGBoostError: [19:03:42] ../src/c_api/c_api_utils.h:159: Invalid missing value: null
Stack trace:
  [bt] (0) /opt/conda/lib/python3.10/site-packages/xgboost/lib/libxgboost.so(+0xbbec9) [0x408d8bbec9]
  [bt] (1) /opt/conda/lib/python3.10/site-packages/xgboost/lib/libxgboost.so(+0xdeb90) [0x408d8deb90]
  [bt] (2) /opt/conda/lib/python3.10/site-packages/xgboost/lib/libxgboost.so(+0xe45d8) [0x408d8e45d8]
  [bt] (3) /opt/conda/lib/python3.10/site-packages/xgboost/lib/libxgboost.so(XGBoosterPredictFromDense+0x330) [0x408d8c44d0]
  [bt] (4) /opt/conda/lib/python3.10/lib-dynload/../../libffi.so.8(+0x6a4a) [0x4002cf0a4a]
  [bt] (5) /opt/conda/lib/python3.10/lib-dynload/../../libffi.so.8(+0x5fea) [0x4002ceffea]
  [bt] (6) /opt/conda/lib/python3.10/lib-dynload/_ctypes.cpython-310-x86_64-linux-gnu.so(+0x12f70) [0x4002cdcf70]
  [bt] (7) /opt/conda/lib/python3.10/lib-dynload/_ctypes.cpython-310-x86_64-linux-gnu.so(+0x12633) [0x4002cdc633]
  [bt] (8) /opt/conda/bin/python(_PyObject_MakeTpCall+0x2ab) 

In [6]:
xgb.plot_tree(clf_xgb)

NameError: name 'xgb' is not defined

In [91]:
!echo $PATH

/opt/conda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin


In [92]:
!which graphviz